##Installation of the libraries

In [1]:
!pip install -q transformers

In [16]:
!pip install biopython

In [17]:
#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch

Torch optimization.

##All libraries needed for training

In [2]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2
import torch.nn.functional as F
# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##Data processing

Getting the sequence of a given file in the target folder (contains only the files with desired sequences).

In [27]:
file_path = "AF-A0A1D8PDI1-F1-model_v4.cif"
file_model = "AF-A0A1D8PDI1-F1-model_v4"
pdbl = PDBList()
pdbl.retrieve_pdb_file(file_path, file_format='mmCif', pdir=".")
# import the needed class
# instantiate the class to prepare the parser
cif_parser = MMCIFParser()
#structure = cif_parser.get_structure("3goe", "3goe.cif")
structure = cif_parser.get_structure(file_model, file_path)
model0 = structure[0]
chain_A = model0['A']  # and we get chain A
# dictionary converting 3-letter codes to 1-letter codes
# this is a very common need in bioinformatics of proteins
d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
 'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
 'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
 'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

sequence = []
for residue in chain_A:
    # for simplicity we can use X for heteroatoms (ions and water)
    sequence.append(d3to1.get(residue.get_resname(), 'X'))  #converts water and ions to X
print(''.join(sequence))

Desired structure doesn't exists
MAHTENYEYPPIPSQQELDDNNVPFFHRDKCAAHLINYYKCLDKGTSFCNKTKDEFYKCQYLALKERLDNHTKQTH


Calculating the angles for the given sequence

In [28]:
#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 1
phi_angles = {}
phi_angles_list = []
psi_angles = {}
psi_angles_list = []

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':
        phi_angles[key] = d[key].angle
        phi_angles_list.append(d[key].angle)
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        psi_angles[key] = d[key].angle
        psi_angles_list.append(d[key].angle)


structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you wan tin the pdb file

Putting angles in a matrix.

In [29]:
phi_angles_list.append(0)
psi_angles_list.append(0)

phi = np.asarray(phi_angles_list,dtype=np.float32)*(np.pi/180)
psi = np.asarray(psi_angles_list,dtype=np.float32)*(np.pi/180)
angles = np.vstack((psi,phi))

Changing sequence for to be used in the Prot-Bert embedding.

In [30]:
seq_example =  ' '.join(sequence)
seq_example

'M A H T E N Y E Y P P I P S Q Q E L D D N N V P F F H R D K C A A H L I N Y Y K C L D K G T S F C N K T K D E F Y K C Q Y L A L K E R L D N H T K Q T H'

##Embedding space creation (using Prot-Bert)

In [31]:
from transformers import BertModel, BertTokenizer
import re
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
seq = re.sub(r"[UZOB]", "X", seq_example)
encoded_input = tokenizer(seq, return_tensors='pt')

##Single-head self unmasked attention layer

In [32]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.w_q = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_k = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_v = nn.Parameter(torch.randn(embed_dim, embed_dim))

    def forward(self, embeddings_prot_bert: torch.Tensor) -> torch.Tensor:
        Q = torch.matmul(embeddings_prot_bert, self.w_q)
        K = torch.matmul(embeddings_prot_bert, self.w_k)
        V = torch.matmul(embeddings_prot_bert, self.w_v)

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attn, V)

        return attention_output

##Encoder with attention and 2 layer FFNN

In [33]:
class TransformerModel(nn.Module):
    def __init__(self, embed_dim: int, feed_forward_dim1: int, feed_forward_dim2: int, output_dim: int = 2, dropout_rate: float = 0.1):
        super(TransformerModel, self).__init__()
        self.self_attention = SelfAttention(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, feed_forward_dim1),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim1, feed_forward_dim2),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim2, output_dim)
        )

    def forward(self, embeddings: torch.Tensor) -> torch.Tensor:
        attention_output = self.self_attention(embeddings)
        normalized_attention_output = self.layer_norm1(attention_output)
        ff_output = self.feed_forward(normalized_attention_output)
        output = self.layer_norm2(ff_output)
        return ff_output


In [34]:
class AngularLoss(nn.Module):
    def __init__(self):
        super(AngularLoss, self).__init__()

    def forward(self, predicted_angles, angles_tensor):
        predicted_angles_phi, predicted_angles_psi = predicted_angles[:, 0], predicted_angles[:, 1]
        angles_tensor_phi, angles_tensor_psi = angles_tensor[:, 0], angles_tensor[:, 1]

        predicted_angles_phi = (predicted_angles_phi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_phi = (angles_tensor_phi + torch.pi) % (2 * torch.pi) - torch.pi
        predicted_angles_psi = (predicted_angles_psi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_psi = (angles_tensor_psi + torch.pi) % (2 * torch.pi) - torch.pi

        difference_phi = torch.abs(predicted_angles_phi - angles_tensor_phi)
        loss_phi = torch.mean(torch.min(difference_phi, 2 * torch.pi - difference_phi))

        difference_psi = torch.abs(predicted_angles_psi - angles_tensor_psi)
        loss_psi = torch.mean(torch.min(difference_psi, 2 * torch.pi - difference_psi))
        
        loss = loss_phi + loss_psi
        print(loss)
        return loss


In [27]:
""" class CustomLoss(nn.Module):
    def __init__(self, predicted_angles, angles_tensor):
        super(CustomLoss, self).__init__()
        self.predicted_angles= predicted_angles
        self.angles_tensor= angles_tensor
    def forward(self):
        print(self.predicted_angles)
        print(self.angles_tensor)
        d_list = []
        for i in range(len(self.angles_tensor)):
            x1, y1 = self.predicted_angles[0][i]
            x2, y2 = self.angles_tensor[i]
            ax_x = torch.min(torch.abs(x2 - x1), torch.abs(360 - torch.abs(x2 - x1)))
            ax_y = torch.min(torch.abs(y2 - y1), torch.abs(360 - torch.abs(y2 - y1)))
            d = torch.sqrt(ax_x**2 + ax_y**2)
            d_list.append(d)
        d_tensor = torch.stack(d_list)
        return d_tensor.mean() """
 

' class CustomLoss(nn.Module):\n    def __init__(self, predicted_angles, angles_tensor):\n        super(CustomLoss, self).__init__()\n        self.predicted_angles= predicted_angles\n        self.angles_tensor= angles_tensor\n    def forward(self):\n        print(self.predicted_angles)\n        print(self.angles_tensor)\n        d_list = []\n        for i in range(len(self.angles_tensor)):\n            x1, y1 = self.predicted_angles[0][i]\n            x2, y2 = self.angles_tensor[i]\n            ax_x = torch.min(torch.abs(x2 - x1), torch.abs(360 - torch.abs(x2 - x1)))\n            ax_y = torch.min(torch.abs(y2 - y1), torch.abs(360 - torch.abs(y2 - y1)))\n            d = torch.sqrt(ax_x**2 + ax_y**2)\n            d_list.append(d)\n        d_tensor = torch.stack(d_list)\n        return d_tensor.mean()\n '

In [40]:
class TransformerTrainer:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: torch.Tensor):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = angles
        self.optimizer = optim.AdamW(model.parameters(), lr=0.001)
        self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        loss_list = []
        for epoch in range(self.num_epochs):
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence)[:, :len(self.angles_tensor)]
            #loss = CustomLoss()
            #loss.forward(torch.tensor(predictions, requires_grad=True), self.angles_tensor)
            loss = self.criterion(predictions.squeeze(), self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            #self.scheduler.step()
            loss_list.append(loss.item())
            #print(f"Epoch {epoch + 1}/{self.num_epochs}, Loss: {loss.item()}")
        return loss



In [29]:
""" def inc_lr(step_size,init_lr,epoch):
    if(epoch % step_size):
        return init_lr + 0.05 """

In [41]:
bert_model = BertModel.from_pretrained("Rostlab/prot_bert")
output = bert_model(**encoded_input)
embedded_pb = output.last_hidden_state
N, D = embedded_pb.size()[1], embedded_pb.size()[2]
embeddings = embedded_pb
# Initialize and train transformer model
feed_forward_dim1 = 512
feed_forward_dim2 = 256
feed_forward_dim3 = 128
num_epochs = 200
dropout_rate = 0.1
model = TransformerModel(embed_dim=D, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2, dropout_rate = dropout_rate)
criterion = AngularLoss()
angles_tensor = torch.from_numpy(angles.T)
trainer = TransformerTrainer(model, criterion, num_epochs, embedded_pb, angles_tensor)
trainer.train()

tensor(4.0447, grad_fn=<AddBackward0>)
tensor(2.7716, grad_fn=<AddBackward0>)
tensor(1.3002, grad_fn=<AddBackward0>)
tensor(1.4943, grad_fn=<AddBackward0>)
tensor(1.0948, grad_fn=<AddBackward0>)
tensor(1.1934, grad_fn=<AddBackward0>)
tensor(1.3014, grad_fn=<AddBackward0>)
tensor(1.1084, grad_fn=<AddBackward0>)
tensor(1.2960, grad_fn=<AddBackward0>)
tensor(1.1951, grad_fn=<AddBackward0>)
tensor(1.1504, grad_fn=<AddBackward0>)
tensor(1.1565, grad_fn=<AddBackward0>)
tensor(1.0683, grad_fn=<AddBackward0>)
tensor(1.1191, grad_fn=<AddBackward0>)
tensor(1.0823, grad_fn=<AddBackward0>)
tensor(1.0971, grad_fn=<AddBackward0>)
tensor(1.0957, grad_fn=<AddBackward0>)
tensor(1.0485, grad_fn=<AddBackward0>)
tensor(1.1256, grad_fn=<AddBackward0>)
tensor(1.0715, grad_fn=<AddBackward0>)
tensor(1.0676, grad_fn=<AddBackward0>)
tensor(1.1275, grad_fn=<AddBackward0>)
tensor(1.0642, grad_fn=<AddBackward0>)
tensor(1.0550, grad_fn=<AddBackward0>)
tensor(1.0301, grad_fn=<AddBackward0>)
tensor(1.0247, grad_fn=<A

tensor(0.5045, grad_fn=<AddBackward0>)

In [42]:
angles_tensor = torch.from_numpy(angles.T)
predicted_angles = model.forward(embedded_pb)[:, :len(angles_tensor)]

In [26]:
""" pred = predicted_angles*(180/np.pi)
torch.save(pred,'predictions/GELU_dr_A0A1D8PD42-F1_200_dyn001_AdamW.pt') """

In [44]:
predicted_angles*(180/np.pi)

tensor([[[158.4100, 109.5332],
         [187.5912, 135.2645],
         [177.7176, 119.8774],
         [161.3522, 104.3600],
         [177.4919, 133.1797],
         [166.9800, 110.2786],
         [176.1008, 111.7786],
         [171.0544, 107.5086],
         [177.1652, 134.9918],
         [183.4278, 126.7361],
         [178.7584, 127.7539],
         [185.5664, 136.2002],
         [177.1126, 113.8967],
         [176.9234, 132.4848],
         [180.2630, -46.0710],
         [188.4490, -43.3479],
         [183.1954, -45.2100],
         [184.9850, -43.7232],
         [159.5721, -33.7733],
         [189.9138, -25.7491],
         [181.4079,   2.2061],
         [180.1419,   4.7576],
         [157.5063, 110.5353],
         [177.8072, -37.4800],
         [170.0987, 124.3076],
         [179.0869, -41.4892],
         [168.6191,   4.9819],
         [177.7564,   7.5654],
         [183.3045, -39.0855],
         [170.2388,   6.3639],
         [184.7458, -42.7739],
         [179.3098, -43.9334],
        

In [39]:
angles_tensor*(180/np.pi)

tensor([[-171.0431,  109.1105],
        [ 174.5438,  119.8184],
        [ 156.8461,  106.0380],
        [ 154.0232,   68.2587],
        [ 174.5967,  112.6536],
        [ 177.0248,   96.2833],
        [ 168.9758,  114.0136],
        [ 168.2972,  115.0865],
        [ 171.5821,  143.6012],
        [ 171.9176,  155.6628],
        [ 176.8698,  135.1075],
        [ 177.1649,  125.2264],
        [ 175.1920,  151.7736],
        [ 176.6922,  155.9873],
        [ 177.7995,  -43.8969],
        [ 177.8293,  -42.8056],
        [ 173.0439,  -34.6937],
        [ 177.3462,  -39.1208],
        [ 179.4949,  -46.1500],
        [ 177.0912,  -33.6513],
        [ 179.9382,   10.9087],
        [-177.4070,   39.4972],
        [ 177.8264,  121.4605],
        [-177.2569,  140.6745],
        [ 177.1559,  -39.4527],
        [ 179.4727,  -26.2765],
        [ 172.6909,    7.2500],
        [-177.5635,   50.7979],
        [ 177.1186, -169.8488],
        [-176.5114,  -15.8719],
        [-177.8203,    1.4454],
        

In [ ]:
structure.atom_to_internal_coordinates() # turns xyz coordinates into angles and bond lengths

chain:PDB.Chain.Chain = list(structure.get_chains())[0]#iterator of chains, turns it into list, [0] first chain

ic_chain: PDB.internal_coords.IC_Chain = chain.internal_coord #this access the internal chain coords of the chain object
#if you modify this, you will modify the orgiginal

d: Dict[Tuple[PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey,
              PDB.internal_coords.AtomKey],
        PDB.internal_coords.Dihedron] = ic_chain.dihedra

cnt = 0
phi_angles = {}
psi_angles = {}

for key in d:
    if key[0].akl[3] == 'N' and key[1].akl[3] == 'CA' and key[2].akl[3] == 'C' and key[3].akl[3] == 'N':

        d[key].angle = angles_tensor[cnt, 0].item()
        #print('phi clculated')
    elif key[0].akl[3] == 'CA' and key[1].akl[3] == 'C' and key[2].akl[3] == 'N' and key[3].akl[3] == 'CA':
        d[key].angle = angles_tensor[cnt, 1].item()
        #print("psi calcukated")
        #print(cnt)
        cnt += 1



structure.internal_to_atom_coordinates(verbose = True)
io = PDBIO() #this is to write a pdb file again
io.set_structure(structure)#set structure, the structure you want in the pdb file
io.save('AF-A0A1D8PD42-F1-model_v4_pred.pdb',  preserve_atom_numbering=True) #saves to a file, filename you a , true - preserves the original atom numbering

In [36]:
angles_tensor*(180/np.pi)

tensor([[ 179.9299,   98.1640],
        [ 166.9430,   82.5050],
        [ 175.7644,   79.1991],
        [ 153.2986,   83.0741],
        [ 139.6692,   60.0443],
        [ 176.9491,  102.7947],
        [ 164.3176,   85.2867],
        [ 178.5258,   42.9345],
        [ 171.2007,  127.9657],
        [ 174.5847,  155.6947],
        [ 173.7650,  138.0246],
        [ 172.4886,  137.1495],
        [ 179.3246,  146.5627],
        [ 170.6717,  -34.0441],
        [-176.4155, -179.8297],
        [-173.8806,   -4.4963],
        [-169.4366,  -13.8653],
        [ 173.5445,  110.5009],
        [ 175.3131,  148.9942],
        [ 167.3701,  154.4301],
        [ 176.2569,  130.1698],
        [ 170.4209,  157.8414],
        [ 171.9277,  157.4808],
        [ 175.1485,  -35.6816],
        [ 175.1972,  -41.2917],
        [ 173.9927,  -40.0640],
        [-179.1476,  -50.0214],
        [-179.6617,  -27.9610],
        [-177.6897,   -0.5788],
        [ 176.5145,  132.7397],
        [ 179.8077,  -31.3434],
        

In [ ]:
class AngularLoss(nn.Module):
    def init(self):
        super(AngularLoss, self).init()

    def forward(self, output, target):
        output_phi, output_psi = output[:, 0], output[:, 1]
        target_phi, target_psi = target[:, 0], target[:, 1]

        # Ensure both angles are in the range (-pi, pi)
        output_phi = (output_phi + torch.pi) % (2 * torch.pi) - torch.pi
        target_phi = (target_phi + torch.pi) % (2 * torch.pi) - torch.pi
        output_psi = (output_psi + torch.pi) % (2 * torch.pi) - torch.pi
        target_psi = (target_psi + torch.pi) % (2 * torch.pi) - torch.pi

        # Compute the differences and wrap them correctly
        difference_phi = torch.abs(output_phi - target_phi)
        loss_phi = torch.mean(torch.min(difference_phi, 2 * torch.pi - difference_phi))

        difference_psi = torch.abs(output_psi - target_psi)
        loss_psi = torch.mean(torch.min(difference_psi, 2 * torch.pi - difference_psi))

        # Combine the losses
        loss = loss_phi + loss_psi
        return loss

def angular_loss_function(output_phi, target_phi, output_psi, target_psi):
    # Ensure both angles are in the range (-pi, pi)
    output_phi = (output_phi + torch.pi) % (2 * torch.pi) - torch.pi
    target_phi = (target_phi + torch.pi) % (2 * torch.pi) - torch.pi
    output_psi = (output_psi + torch.pi) % (2 * torch.pi) - torch.pi
    target_psi = (target_psi + torch.pi) % (2 * torch.pi) - torch.pi
    
    # Compute the differences and wrap them correctly
    difference_phi = torch.abs(output_phi - target_phi)
    loss_phi = torch.mean(torch.min(difference_phi, 2 * torch.pi - difference_phi))

    difference_psi = torch.abs(output_psi - target_psi)
    loss_psi = torch.mean(torch.min(difference_psi, 2 * torch.pi - difference_psi))

    # Combine the losses
    loss = loss_phi + loss_psi
    return loss